In [9]:
from langchain_ollama import ChatOllama

model = ChatOllama(
    model="qwen3:1.7b",
    temperature=0.8
)

response = model.invoke("用中文解释机器学习")
print("本地模型回复:", response.content)

本地模型回复: 机器学习是人工智能的一个分支，它的核心目标是让计算机通过数据学习规律，从而完成特定任务。简单来说，机器学习就是让计算机“自己学习”，而不是依靠人直接编程。

### 机器学习的核心概念：
1. **数据**：机器学习需要大量的数据作为“老师”，数据中包含需要学习的规律或特征。
2. **模型**：模型是计算机用来“思考”的结构，比如线性回归、分类器等。
3. **训练过程**：通过调整模型的参数，让模型从数据中找到规律，这个过程称为训练。

### 机器学习的类型：
1. **监督学习**：  
   - 用带标签的数据（比如“这是狗”）训练模型，模型学会从输入数据到输出标签的映射关系。  
   - 常见任务：分类（如垃圾邮件检测）、回归（如预测房价）。

2. **无监督学习**：  
   - 不用标签数据，直接从数据中发现规律。  
   - 常见任务：聚类（如客户分群）、降维（如数据压缩）。

3. **强化学习**：  
   - 通过试错的方式学习策略，比如机器人学习走路。  
   - 模型需要与环境互动，通过奖励机制优化行为。

### 举例说明：
- **垃圾邮件检测**：模型通过分析邮件内容（特征），学会识别哪些邮件是垃圾的。  
- **推荐系统**：根据用户的历史行为（数据）训练模型，预测用户可能喜欢的物品。  
- **人脸识别**：模型从大量人脸图像中学习特征，识别新图像中的面孔。

### 机器学习的挑战：
- **需要大量数据**：数据质量直接影响效果。  
- **依赖数据质量**：噪声或偏差会误导模型。  
- **模型可解释性**：有些复杂模型（如深度学习）难以解释，限制实际应用。

### 总结：
机器学习就像“老师教学生”，但老师是计算机，学生是数据，最终目标是让计算机“自己学会解决问题”。它广泛应用于推荐系统、医疗诊断、自动驾驶等领域，是人工智能的核心技术之一。


In [2]:
from langchain.tools import tool

@tool
def get_weather(location: str) -> str:
    """获取指定位置的天气信息。"""
    return f"{location}的天气：晴朗，25°C"

@tool
def calculator(expression: str) -> str:
    """计算数学表达式。"""
    try:
        result = eval(expression)
        return f"{expression} = {result}"
    except:
        return "计算错误"

In [3]:
model = model.bind_tools([get_weather,calculator])
response = model.invoke("北京今天天气怎么样？然后计算 25 * 4")
print("工具调用：",response.tool_calls)

工具调用： [{'name': 'get_weather', 'args': {'location': '北京'}, 'id': '6159028f-dcf9-41fa-b18d-2d8afa0b7386', 'type': 'tool_call'}, {'name': 'calculator', 'args': {'expression': '25 * 4 /'}, 'id': '2df3cc73-c9cc-4b60-8e9f-636dbede2288', 'type': 'tool_call'}]


In [4]:
response

AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen3:1.7b', 'created_at': '2025-10-22T05:51:16.3606361Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3179999800, 'load_duration': 72092000, 'prompt_eval_count': 192, 'prompt_eval_duration': 179271800, 'eval_count': 389, 'eval_duration': 2810439100, 'model_name': 'qwen3:1.7b', 'model_provider': 'ollama'}, id='lc_run--6bcac2d2-4406-4876-8958-5667a3181b06-0', tool_calls=[{'name': 'get_weather', 'args': {'location': '北京'}, 'id': '6159028f-dcf9-41fa-b18d-2d8afa0b7386', 'type': 'tool_call'}, {'name': 'calculator', 'args': {'expression': '25 * 4 /'}, 'id': '2df3cc73-c9cc-4b60-8e9f-636dbede2288', 'type': 'tool_call'}], usage_metadata={'input_tokens': 192, 'output_tokens': 389, 'total_tokens': 581})

In [8]:
from typing import Optional,List
from pydantic import BaseModel,Field

class Actor(BaseModel):
    """演员信息"""
    name: str = Field(description="演员姓名")
    character: str = Field(description="扮演角色")

class MovieDetails(BaseModel):
    """详细电影信息"""
    title: str = Field(description="电影标题")
    year: int = Field(description="上映年份")
    director: str = Field(description="导演")
    cast: List[Actor] = Field(description="演员表")
    budget: Optional[float] = Field(description="预算（百万美元）")
    box_office: Optional[float] = Field(description="票房（百万美元）")

# 使用嵌套结构
detailed_model = model.with_structured_output(MovieDetails)
response = detailed_model.invoke("提供《泰坦尼克号》的完整信息")

print(f"电影: {response.title} ({response.year})")
print(f"导演: {response.director}")
print("主演:")
for actor in response.cast:
    print(f"  - {actor.name} 饰演 {actor.character}")

电影: 泰坦尼克号 (1997)
导演: 詹姆斯·卡梅隆
主演:
  - 莱昂纳多·迪卡普里奥 饰演 卡尔·霍克
  - 凯特·布兰切特 饰演 露丝·伯恩斯
